In [1]:
import bs4 as bs
import pickle
import requests
import datetime as dt
import pandas as pd
import pandas_datareader.data as pdr
import os
import yfinance as yf

yf.pdr_override

<function yfinance.pdr_override()>

In [2]:
def save_sp500_tickers():
    resp = requests.get('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
    soup = bs.BeautifulSoup(resp.text)
    table = soup.find('table', {'class': 'wikitable sortable'})
    tickers = []
    for row in table.findAll('tr')[1:]:
        ticker = row.findAll('td')[0].text.replace('.', '-')
        ticker = ticker[:-1]
        tickers.append(ticker)
    
    with open('sp500tickers.pickle', 'wb') as f:
        pickle.dump(tickers, f)
    
    #print(tickers)
    
    return tickers

In [ ]:
save_sp500_tickers()

In [5]:
def get_data_from_yahoo(reloadSP500=False):
    if reloadSP500: 
        tickers = save_sp500_tickers()
    else:
        with open('sp500tickers.pickle', 'rb') as f:
            tickers = pickle.load(f)
    
    if not os.path.exists('stocks_dfs'):
        os.makedirs('stocks_dfs', exist_ok=True)
    
    start = dt.datetime(2019,6,8)
    end = dt.datetime.now()
    
    for ticker in tickers[0:10]:
        print(ticker)
            
        if not os.path.exists('stocks_dfs/{}.csv'.format(ticker)):
            df = pdr.get_data_yahoo(ticker, start, end)
            df.reset_index(inplace=True)
            df.set_index('Date', inplace=True)
            df.to_csv('stocks_dfs/{}.csv'.format(ticker))
        else:
            print('Already have {}'.format(ticker))

In [7]:
get_data_from_yahoo()

MMM
Already have MMM
ABT


ConnectTimeout: HTTPSConnectionPool(host='finance.yahoo.com', port=443): Max retries exceeded with url: /quote/ABT/history?period1=1559962800&period2=1610247599&interval=1d&frequency=1d&filter=history (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x119f63460>, 'Connection to finance.yahoo.com timed out. (connect timeout=30)'))